In [1]:
# Instalación de dependencias
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import evaluate

In [2]:
# Carga de modelos y tokenizadores
t5_model_name = "t5-small"
gpt2_model_name = "gpt2"

In [3]:
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

C:\Users\epuerta\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\epuerta\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_model_name)
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\epuerta\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\epuerta\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
# Texto de entrada y referencia
texto_entrada = "The Eiffel Tower is located in Paris and was built in 1889."
referencia_resumen = "The Eiffel Tower was built in Paris in 1889."

In [6]:
# Generación con T5
entrada_t5 = "summarize: " + texto_entrada
inputs_t5 = t5_tokenizer(entrada_t5, return_tensors="pt")
output_t5 = t5_model.generate(**inputs_t5, max_length=30, temperature=0.7)
texto_t5 = t5_tokenizer.decode(output_t5[0], skip_special_tokens=True)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [7]:
# Generación con GPT-2
inputs_gpt2 = gpt2_tokenizer(texto_entrada, return_tensors="pt")
output_gpt2 = gpt2_model.generate(**inputs_gpt2, max_length=30, temperature=0.7,
do_sample=True, top_k=50)
texto_gpt2 = gpt2_tokenizer.decode(output_gpt2[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [8]:
# Evaluación con BLEU y ROUGE
bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")

In [9]:
resultado_bleu_t5 = bleu.compute(predictions=[texto_t5], references=[[referencia_resumen]])
resultado_rouge_t5 = rouge.compute(predictions=[texto_t5],
references=[referencia_resumen])

In [10]:
print("T5 Generado:", texto_t5)
print("BLEU T5:", resultado_bleu_t5)
print("ROUGE T5:", resultado_rouge_t5)

T5 Generado: the Eiffel Tower was built in 1889.
BLEU T5: {'score': 52.08154731870474, 'counts': [7, 6, 4, 2], 'totals': [8, 7, 6, 5], 'precisions': [87.5, 85.71428571428571, 66.66666666666667, 40.0], 'bp': 0.7788007830714049, 'sys_len': 8, 'ref_len': 10}
ROUGE T5: {'rouge1': np.float64(0.8750000000000001), 'rouge2': np.float64(0.8571428571428571), 'rougeL': np.float64(0.8750000000000001), 'rougeLsum': np.float64(0.8750000000000001)}


In [22]:
# Perplexity para GPT-2
import math
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [23]:
# Cargar el modelo y el tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [25]:
# Función para calcular la perplexity
def calcular_perplexity(model, tokenizer, texto):
    encodings = tokenizer(texto, return_tensors="pt")
    input_ids = encodings.input_ids
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        loss = outputs.loss
        ppl = math.exp(loss.item())
    return ppl

# Texto de ejemplo
texto_gpt2 = "El sol brilla sobre las montañas y el viento sopla suavemente."
print("GPT-2 Generado:", texto_gpt2)
print("Perplexity GPT-2:", calcular_perplexity(model, tokenizer, texto_gpt2))

GPT-2 Generado: El sol brilla sobre las montañas y el viento sopla suavemente.
Perplexity GPT-2: 261.38433668058985
